# Plot the data
Used to map the COVID-19 cases of Germany on a map or plotting the number of cases/seven days incidence of one county.
<br/><br/><br/>
Provides the dictionary "counties_geography" with different kinds of geographical information about every German county. For more information check out the file "get_geographical_data_of_german_counties.ipynb".
<br/><br/>
Provides the accumulated COVID-19 case number and the seven days incidences of every german county and every day of the pandemic in the dictionary "covid19" (reachable by the county's AdmUnitID).
For more information check out the file "get_data.ipynb" (which calls the file "get_geographical_data_of_german_counties.ipynb" if needed).

## Modules
Needed to use non-Python functionalities already programmed by someone else.

In [1]:
# Used to cast the polygons into np.arrays and afterwards transpose them easily with the .T function
import numpy as np
import matplotlib.pyplot as plt    # to plot the counties
# Used to determine which part of the plot shall be saved
from matplotlib.transforms import Bbox
# Used to underline text in plots
from matplotlib import rc

## Get the data
More information about the data can be find inside the first cell.

In [2]:
%run get_data.ipynb
print("All data is ready!")

Polished county data from file is ready to go!
Polished covid19 data from file is ready to go!
All data is ready!


## Create color schemes

In [3]:
# check if the needed directory is availlable - otherwise create it
if not(os.path.isdir("media")): os.makedirs("media")
# check if the needed directory is availlable - otherwise create it
if not(os.path.isdir("media/germany_incidence_V4")): os.makedirs("media/germany_incidence_V4")

In [4]:
red_yellow_green = list()
# bigger steps in the darkgreen spectrum to make a bigger optical difference between
# smaller numbers (e.g. 7 and 8) in comparison to bigger numbers (e.g. 57 and 58)
# (most of the time most counties are green and hard to distinguish and only a few are red)
for color_step in np.arange(0.4,1,0.05):
    red_yellow_green.append((0, color_step, 0))
for color_step in np.arange(0,1,0.02):
    red_yellow_green.append((color_step, 1, 0))
for color_step in np.arange(0,1,0.01):
    red_yellow_green.append((1, 1 - color_step, 0))
for color_step in np.arange(0,0.94,0.005):
    red_yellow_green.append((1 - color_step, 0, 0))

In [5]:
def plot_map_of_germany_incidences(date_index, fig, ax):
    for AdmUnitID, county in covid19.items():
        color_index_incidence = int(county['incidences'][date_index])
        if color_index_incidence >= len(red_yellow_green):
            color = red_yellow_green[-1]
        else:
            color = red_yellow_green[color_index_incidence]
        for polygon in counties_geography[AdmUnitID]['geometry']:
            x,y = np.array(polygon).T
            plt.fill(x, y, color=color, facecolor="darkgrey")
        # draw the lines around the counties
        for polygon in counties_geography[AdmUnitID]['raw_geometry']:
            x,y = np.array(polygon).T
            plt.plot(x, y, color="black")

In [6]:
def plot_legend_incidences():
    ax1 = plt.axes([0.01, 0.2, 0.06, 0.65], # [left, bottom, width, height]
                   frameon=False)
    ax1.yaxis.tick_right()
    ax1.axes.get_xaxis().set_visible(False)
    plt.yticks([0,25,50,75,100,125,150,175,200,225,250,275,300,325,350],
               ["0","25","50","75","100","125","150","175","200","225","250","275","300","325",">350"],
               fontsize=90)
    plt.ylabel('Seven-days-incidence', fontsize=100, rotation='vertical')
    for color_index in range(len(red_yellow_green)):
        plt.fill([1, 2, 2, 1], [1+color_index, 1+color_index, 2+color_index, 2+color_index],
                 color=red_yellow_green[color_index])
        if color_index % 25 == 0:
            plt.fill([1, 2, 2, 1], [color_index, color_index, 1+color_index, 1+color_index],
                     color="black")
    plt.fill([1, 2, 2, 1], [len(red_yellow_green), len(red_yellow_green), 10+len(red_yellow_green), 10+len(red_yellow_green)],
            color="black")

In [7]:
def plot_distribution_of_incidences(date_index):
    ax2 = plt.axes([0.96, 0.31, 0.55, 0.36], facecolor='lightgrey') # [left, bottom, width, height]
    plt.xticks([num for num in range(0,351,50)]+[375],
               [str(num) for num in range(0,351,50)] + 
               [str(int(non_county_specific_data["highest_incidence"]))],
              fontsize=70)
    plt.yticks([10,20,30,40,50,60,70,80,90,100,110,120,130,140,150], fontsize=70)
    plt.xlim(0, 350)
    plt.ylim(0, 150)
    plt.title(r"\textbf{Number of counties with given incidence}",
              fontsize=90, usetex=True, y=1.02, x=0.55)
    ax2.set_ylabel("Number of counties", fontsize=75)
    plt.xlabel("Incidence", fontsize=70)
    _, bins, patches = ax2.hist([county['incidences'][date_index] for county in covid19.values()],
             bins=[e for e in range(0,351,25)]+[350, 100000], color="darkslategrey",
                                edgecolor="black")
    bin_centers = 0.5*(bins[:-1]+bins[1:])
    col = bin_centers - min(bin_centers)
    red_yellow_green_index=0
    for c, p in zip(col, patches):
        if len(red_yellow_green) <= red_yellow_green_index:
            plt.setp(p, "facecolor", "black")
        else:
            plt.setp(p, "facecolor", red_yellow_green[red_yellow_green_index])
        red_yellow_green_index += 25
    plot_distribution_of_incidences_over_350(date_index)

In [8]:
def plot_distribution_of_incidences_over_350(date_index):
    ax4 = plt.axes([1.55, 0.31, 0.04, 0.36], facecolor='lightgrey') # [left, bottom, width, height]
    ax4.axes.get_yaxis().set_visible(False)
    plt.xticks([375],[">350"],fontsize=70)
    plt.xlim(350, 400)
    plt.ylim(0, 150)
    _, bins, patches = ax4.hist([county['incidences'][date_index] for county in covid19.values()],
             bins=[0, 350, 100000], color="darkslategrey", edgecolor="black")
    bin_centers = 0.5*(bins[:-1]+bins[1:])
    col = bin_centers - min(bin_centers)
    for c, p in zip(col, patches):
        plt.setp(p, "facecolor", "black")

In [9]:
for date_index in range(0,8):# len(non_county_specific_data['UTC'])):
    fig = plt.figure(figsize = (50, 65), facecolor='darkgrey')
    ax = plt.axes(frameon=False)
    ax.axes.get_xaxis().set_visible(False)
    ax.axes.get_yaxis().set_visible(False)
    plot_map_of_germany_incidences(date_index, fig, ax)
    plot_distribution_of_incidences(date_index)
    plot_legend_incidences()
    plt.text(0.5, -25, non_county_specific_data['UTC+7days'][date_index] + " to ",
             fontsize=80)
    plt.text(0.5, -39, non_county_specific_data['UTC+7days'][date_index + 7],
             fontsize=80)
    plt.text(17.5, 337, r'\textbf{\underline{Seven-days-incidence in Germany}}',
              fontweight=220, fontsize=170, usetex=True)
    plt.text(17.5, 317, "The COVID-19 cases of every German county", fontsize=110)
    plt.text(17.5, 297, "per week scaled to 100.000 people.", fontsize=110)
    plt.text(17.5, 12, "Data from the COVID-19 Datenhub referenced by the Robert-Koch-Institut (www.rki.de):",
             fontsize=55)
    plt.text(17.5, 2, 'npgeo-corona-npgeo-de.hub.arcgis.com (Used "RKI History" and "RKI Corona Landkreise")',
             fontsize=55)
    plt.text(17.5, -13, 'Created by Leander Bürkin (leander.buerkin@gmail.com),',
             fontsize=55)
    plt.text(17.5, -23, 'Laboratory for Simulation, Prof. Dr. Lars Pastewka, Albert-Ludwigs-Universität Freiburg',
             fontsize=55)
    plt.text(17.5, -33, 'https://github.com/leanderbuerkin/germany_model_reconstruction',
             fontsize=55)
    plt.savefig("media/germany_incidence_V4/" + str(date_index) + '.png',
                facecolor="darkgrey", bbox_inches=Bbox([[-3.5,9],[84,57]]))
    plt.close('all')

In [10]:
g = list()
g[1]

IndexError: list index out of range

## Plot examples

In [ ]:
# copy to check that all counties are related to a federal state
covid19_copy = covid19.copy()
for stateID, state in non_county_specific_data['states'].items():
    plt.figure(figsize=(20, 5))    # enlarge plot
    plt.title("Incidences in {} {}".format(stateID, state))
    plt.xticks(non_county_specific_data["unixtime"][::7],
               non_county_specific_data['UTC'][::7], rotation='vertical')
    for AdmUnitID, county in covid19.items():
        if AdmUnitID[:-3] == stateID:
            plt.plot(non_county_specific_data["unixtime"], county['incidences'])
            del covid19_copy[AdmUnitID]
    plt.show()

In [ ]:
if len(covid19_copy) != 0:
    print('There seems to be at least one county being unrelated to any federal state:')
    for AdmUnitID in covid19_copy.keys():
        print(AdmUnitID + " " + counties_geography[AdmUnitId]['name'])

In [ ]:
acc = list()
for e in [county['incidences'] for county in covid19.values()]:
    for i in e:
        if i>100 and i<101:
            acc.append(i)
acc.sort()
plt.yticks([350,400])
plt.title(str(len(acc)))
plt.plot(acc)
plt.show()

## Plot data of one state

In [ ]:
example_state_ID = "13"
print("You have chosen {} {}".format(example_state_ID, non_county_specific_data['states'][example_state_ID]))

In [ ]:
counties_in_example_state = list()
for AdmUnitID, county in covid19.items():
    if AdmUnitID[:-3] == "13":
        counties_in_example_state.append(AdmUnitID)
        plt.figure(figsize=(20, 5))    # enlarge plot
        plt.xticks(non_county_specific_data["unixtime"][::7],
                   non_county_specific_data['UTC'][::7], rotation='vertical')
        plt.plot(non_county_specific_data["unixtime"], county['incidences'])
        plt.title("Incidences in " + AdmUnitID + " " + counties_geography[AdmUnitID]['name'])
        plt.show()

In [ ]:
# plt.figure(figsize = (20, 20))    # enlarge plot
for AdmUnitID in counties_in_example_state:
    # get last incidence and divide it by the highest incidence to get a number between 0 and 1
    # to get the full range of colors the both incidence values get subtracted by the lowest incidence
    color_step = covid19[AdmUnitID]['incidences'][-1]/250
    # everything bellow 0.5 gets drawn green
    if color_step > 1:
        color = (1, 1, 0)
    elif color_step < 0.2:
        color = (color_step * 5, 1, 0)
    else:
        color_step = (color_step - 0.2) * 1.25
        color = (1, 1 - color_step, 0)
    for polygon in counties_geography[AdmUnitID]['geometry']:
        x,y = np.array(polygon).T
        plt.fill(x, y, color=color)
plt.show()